# Primera apoximación para resolver la pregunta 1

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, countDistinct
spark = SparkSession.builder \
    .remote("sc://localhost:15002").getOrCreate()

## Versión DF

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DateType

schema = StructType([
    StructField("video_id", StringType(), True),
    StructField("title", StringType(), True),
    StructField("publishedAt", DateType(), True),
    StructField("channelId", StringType(), True),
    StructField("channelTitle", StringType(), True),
    StructField("categoyoryId", StringType(), True),
    StructField("trending_date", DateType(), True),
    StructField("tags", StringType(), True),
    StructField("view_count", IntegerType(), True),
    StructField("likes", IntegerType(), True),
    StructField("dislikes", IntegerType(), True),
    StructField("comment_count", IntegerType(), True),
    StructField("thumbnail_link", StringType(), True),
    StructField("comments_disabled", StringType(), True),
    StructField("ratings_disabled", StringType(), True),
    StructField("rank", IntegerType(), True),
])
ranked_videos = spark.read.schema(schema).option("delimiter", "\t").csv("/user/hadoop2/BR_ranked/BR")

ranked_videos.dropna(subset=['title'])
ranked_videos.show()

+-----------+--------------------+-----------+--------------------+--------------------+------------+-------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+----+
|   video_id|               title|publishedAt|           channelId|        channelTitle|categoyoryId|trending_date|                tags|view_count| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|rank|
+-----------+--------------------+-----------+--------------------+--------------------+------------+-------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+----+
|s9FH4rDMvds|LEVEI UM FORA? FI...| 2020-08-11|UCGfBwrCoi9ZJjKiU...|       Pietro Guedes|          22|   2020-08-12|pietro|guedes|ing...|    263835| 85095|     487|         4500|https://i.ytimg.c...|            False|           False|   1|
|jbGRowa5tIk|ITZY “Not Shy” M/...| 2020-08-1

In [3]:
# categorias por canal
categorias_por_canal = ranked_videos.groupBy("channelId", "channelTitle") \
    .agg(countDistinct("categoyoryId").alias("distinct_category_count")) \
    .orderBy(col("distinct_category_count").desc())
    
categorias_por_canal.show()

+--------------------+------------------+-----------------------+
|           channelId|      channelTitle|distinct_category_count|
+--------------------+------------------+-----------------------+
|UCUN9lhwfMJRxMVue...|    YouTube Brasil|                      6|
|UCuxfOdbKQy0tgGXc...|     Coisa de Nerd|                      5|
|UC0otZdGYsA9KqVKA...|         Brancoala|                      5|
|UCQ8a1n0D1PjI8fxR...|          Talokudo|                      4|
|UC2bYhAHyaqfWlPXW...|         Enaldinho|                      4|
|UCWKtHaeXVzUscYGc...|           BRKsEDU|                      4|
|UCLXl1V6n82Dyg1Vh...|  Aviões e Músicas|                      4|
|UColnnUYyz7CWjhzp...|        NOW UNITED|                      4|
|UCzIJhiubPzLf3y6c...|   Thiago da Achah|                      4|
|UCnYjd7cPNluC7Hi9...|      LOUD Coringa|                      4|
|UCKe5eBtF9zbGmPuq...|       Warner Play|                      3|
|UCL-ce0o3Uoj_v6x-...|      Conexão Teen|                      3|
|UCxgO9Tos

In [4]:
# videos por canal
top_videos_canal = ranked_videos.filter(col("rank") <= 50) \
.groupBy("channelId", "trending_date") \
.agg(count("*").alias("top_videos_count"))



top_videos_canal.show()

+--------------------+-------------+----------------+
|           channelId|trending_date|top_videos_count|
+--------------------+-------------+----------------+
|UC-8Q-hLdECwQmaWN...|   2020-08-15|               1|
|UCdkhkda5VQPbKSzx...|   2020-08-19|               1|
|UCgSvw7asvNq7wJWK...|   2020-08-24|               1|
|UCz2A8OXLR2xwYE63...|   2020-08-27|               1|
|UC1H-sZk-cj-tDpfT...|   2020-08-31|               1|
|UCY1zV0j71X2H1H1I...|   2020-09-08|               1|
|UC32z4mtyiq02Ge-X...|   2020-09-24|               1|
|UC0LlPl_mWhIFqVxc...|   2020-09-26|               1|
|UCTv-XvfzLX3i4IGW...|   2020-09-28|               1|
|UC9OMsuaqvGdF0MtK...|   2020-10-01|               1|
|UCi_6SNdvEOq5i1X_...|   2020-10-01|               1|
|UCfIXdjDQH9Fau7y9...|   2020-10-02|               1|
|UCy-pRSIMMBsp-AZl...|   2020-10-13|               1|
|UC_XGgRFoktKUXZVD...|   2020-10-17|               1|
|UC9jMFcUztj-dAulU...|   2020-10-17|               1|
|UC9mdw2mmn49ZuqGO...|   202

In [5]:
resultado = categorias_por_canal.join(
    top_videos_canal,
    categorias_por_canal["channelId"] == top_videos_canal["channelId"]
).select(
    top_videos_canal["trending_date"],
    categorias_por_canal["channelId"],
    categorias_por_canal["channelTitle"],
    categorias_por_canal["distinct_category_count"],
    top_videos_canal["top_videos_count"]
).orderBy(col("top_videos_count").desc())


# Mostrar el resultado
resultado.show(truncate=False)


+-------------+------------------------+-----------------+-----------------------+----------------+
|trending_date|channelId               |channelTitle     |distinct_category_count|top_videos_count|
+-------------+------------------------+-----------------+-----------------------+----------------+
|2024-02-12   |UCqRraVICLr0asn90cAvkIZQ|Corinthians TV   |1                      |3               |
|2023-11-05   |UCw5-xj3AKqEizC7MvHaIPqA|FOX Sports Brasil|2                      |3               |
|2024-03-12   |UCw5-xj3AKqEizC7MvHaIPqA|ESPN Brasil      |1                      |3               |
|2022-09-05   |UCw5-xj3AKqEizC7MvHaIPqA|ESPN Brasil      |1                      |3               |
|2024-03-12   |UCw5-xj3AKqEizC7MvHaIPqA|FOX Sports Brasil|2                      |3               |
|2022-10-07   |UCw4YGFtvDgOL09KrFpqj6kg|A Fazenda        |1                      |3               |
|2022-09-02   |UCw5-xj3AKqEizC7MvHaIPqA|ESPN Brasil      |1                      |3               |


## Versión SQL

In [6]:
ranked_videos.createOrReplaceTempView("ranked_videos")

query = """
    SELECT 
        channelId,
        channelTitle,
        COUNT(DISTINCT categoyoryId) AS distinct_category_count
    FROM 
        ranked_videos
    GROUP BY 
        channelId,
        channelTitle
    ORDER BY 
        distinct_category_count DESC
"""

# categorias por canal
categorias_por_canal = spark.sql(query)
    
categorias_por_canal.show()

+--------------------+------------------+-----------------------+
|           channelId|      channelTitle|distinct_category_count|
+--------------------+------------------+-----------------------+
|UCUN9lhwfMJRxMVue...|    YouTube Brasil|                      6|
|UCuxfOdbKQy0tgGXc...|     Coisa de Nerd|                      5|
|UC0otZdGYsA9KqVKA...|         Brancoala|                      5|
|UCQ8a1n0D1PjI8fxR...|          Talokudo|                      4|
|UC2bYhAHyaqfWlPXW...|         Enaldinho|                      4|
|UCWKtHaeXVzUscYGc...|           BRKsEDU|                      4|
|UCLXl1V6n82Dyg1Vh...|  Aviões e Músicas|                      4|
|UColnnUYyz7CWjhzp...|        NOW UNITED|                      4|
|UCzIJhiubPzLf3y6c...|   Thiago da Achah|                      4|
|UCnYjd7cPNluC7Hi9...|      LOUD Coringa|                      4|
|UCKe5eBtF9zbGmPuq...|       Warner Play|                      3|
|UCL-ce0o3Uoj_v6x-...|      Conexão Teen|                      3|
|UCxgO9Tos

In [7]:
query = """
    SELECT 
        channelId,
        trending_date,
        COUNT(*) AS top_videos_count
    FROM 
        ranked_videos
    WHERE 
        rank <= 50
    GROUP BY 
        channelId, trending_date
"""

# categorias por canal
top_videos_canal = spark.sql(query)
    
top_videos_canal.show()

+--------------------+-------------+----------------+
|           channelId|trending_date|top_videos_count|
+--------------------+-------------+----------------+
|UC-8Q-hLdECwQmaWN...|   2020-08-15|               1|
|UCdkhkda5VQPbKSzx...|   2020-08-19|               1|
|UCgSvw7asvNq7wJWK...|   2020-08-24|               1|
|UCz2A8OXLR2xwYE63...|   2020-08-27|               1|
|UC1H-sZk-cj-tDpfT...|   2020-08-31|               1|
|UCY1zV0j71X2H1H1I...|   2020-09-08|               1|
|UC32z4mtyiq02Ge-X...|   2020-09-24|               1|
|UC0LlPl_mWhIFqVxc...|   2020-09-26|               1|
|UCTv-XvfzLX3i4IGW...|   2020-09-28|               1|
|UC9OMsuaqvGdF0MtK...|   2020-10-01|               1|
|UCi_6SNdvEOq5i1X_...|   2020-10-01|               1|
|UCfIXdjDQH9Fau7y9...|   2020-10-02|               1|
|UCy-pRSIMMBsp-AZl...|   2020-10-13|               1|
|UC_XGgRFoktKUXZVD...|   2020-10-17|               1|
|UC9jMFcUztj-dAulU...|   2020-10-17|               1|
|UC9mdw2mmn49ZuqGO...|   202

In [9]:
categorias_por_canal.createOrReplaceTempView("categorias_por_canal")
top_videos_canal.createOrReplaceTempView("top_videos_canal")

query = """
SELECT 
    tvc.trending_date,
    cpc.channelId,
    cpc.channelTitle,
    cpc.distinct_category_count,
    tvc.top_videos_count
FROM 
    categorias_por_canal cpc
JOIN 
    top_videos_canal tvc
ON 
    cpc.channelId = tvc.channelId
ORDER BY 
    tvc.top_videos_count DESC
"""

# categorias por canal
resultado = spark.sql(query)

resultado.show(truncate=False)

+-------------+------------------------+-------------------------------------------+-----------------------+----------------+
|trending_date|channelId               |channelTitle                               |distinct_category_count|top_videos_count|
+-------------+------------------------+-------------------------------------------+-----------------------+----------------+
|2021-07-29   |UC-FQUIVQ-bZiefzBiQAa8Fw|BBC Radio 1                                |1                      |3               |
|2020-10-30   |UCw4YGFtvDgOL09KrFpqj6kg|A Fazenda                                  |1                      |3               |
|2021-10-15   |UCgCKagVhzGnZcuP9bSMgMCg|ge                                         |1                      |3               |
|2021-09-07   |UCiQ7Ll2Gsb5eQhgQQhZ2lGQ|MFM - Esporte Interativo                   |1                      |3               |
|2021-11-25   |UCs-6sCz2LJm1PrWQN4ErsPw|Esporte Interativo                         |1                      |3         

# Segunda aproximación para resolver la pregunta 1